In [1]:
import requests
import pandas as pd
import json
import time
import mysql.connector

from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from datetime import datetime
from sqlalchemy import create_engine
from collections import Counter
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [2]:
def mysql_to_df():
    mydb = mysql.connector.connect(
            host="",
            user="",
            passwd="",
            database=""
    )

    mycursor = mydb.cursor()
    mycursor.execute("SELECT ms.id,ms.lid,ms.lname,ms.hname,ms.aname,ms.sportName,ms.time,ms.end_time,acc.nickname,acc.id  from smtv.match_schedules AS ms JOIN smtv.match_schedule_rooms AS msr ON msr.sch_id = ms.id AND (( ms.time between '2020-06-25 08:00:00' AND '2020-06-25 20:00:00' ) or ( ms.time between '2020-06-26 08:00:00' AND '2020-06-26 20:00:00' ) or ( ms.time between '2020-06-27 08:00:00' AND '2020-06-27 20:00:00' ) or ( ms.time between '2020-06-28 08:00:00' AND '2020-06-28 20:00:00' )) JOIN smtv.accounts AS acc ON acc.source !=3 AND acc.id = msr.room_id;")
    myresult = mycursor.fetchall()
    df = pd.DataFrame(myresult)

    mycursor.close()
    mydb.close()
    
    return df

df = mysql_to_df()

id_list = []
lname_list = []
hname_list = []
aname_list = []
sportName_list = []
nickname_list = []

for i in range(len(df[0])):
    id_list.append(df[9][i])
    lname_list.append(df[2][i])
    hname_list.append(df[3][i])
    aname_list.append(df[4][i])
    sportName_list.append(df[5][i])
    nickname_list.append(df[8][i])

mobileEmulation = {'deviceName': 'iPhone X'}

options = webdriver.ChromeOptions()

options.add_experimental_option('mobileEmulation', mobileEmulation)

def Smtv_ios():
    error_list = []
    for room_id,lname,hname,aname,sportName,nickname in zip(id_list,lname_list,hname_list,aname_list,sportName_list,nickname_list):
        try:
            id_url = "https://smtv.io/room/"+str(room_id)
            CHROMEDRIVER_PATH = './tools/chromedriver.exe'
            driver = webdriver.Chrome(CHROMEDRIVER_PATH, chrome_options=options)
            driver.get(id_url)
            driver.maximize_window()
            time.sleep(1)
            local_date = datetime.now().strftime('%Y%m%d%H%M')
            driver.save_screenshot( "./data_ios/"+ local_date +"-"+ str(room_id) +"-"+ nickname +"-"+ sportName +"-"+ lname +"-"+ hname +"-"+aname+".png")
            driver.close()
#             print("done: ", room_id," - ",nickname)
            
        except:
            error_dict = {
                "type" : "ios",
                "room_id" : room_id,
                "nickname":nickname,
                "datetime":local_date
                
            }
            error_list.append(error_dict)
            
    error_df = pd.DataFrame(error_list,columns=["type","room_id","nickname","datetime"])
    error_df.to_csv("./error.csv",encoding="utf_8_sig",mode='a',header=False,index=False)
        
Smtv_ios()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: use options instead of chrome_options
